In [ ]:
import math

class PdbReader:
  def __init__(self,filepath):
    self.filepath = filepath
    self.atom_cordinate = []
    self.hetero_cordinate = []
    self.store = {}
    self.atom_list = []
    self.hetero_atom = set()

    with open(filepath,"r") as file:
      for line in file:
        if len(line) >= 54:
          try:
            xcord = float(line[30:38].strip())
            ycord = float(line[38:46].strip())
            zcord = float(line[46:54].strip())
            resid = int(line[22:26])
            resname = line[17:20]
            atom_type = line[0:7].strip()
            atomname = line[11:16].strip()
          except ValueError:
            print(f"Skipping line due to invalid coordinate data: {line}")
            continue

          if line.startswith("ATOM"):
            self.atom_cordinate.append([xcord,ycord,zcord])
          elif line.startswith("HETATM"):
            self.hetero_cordinate.append([xcord,ycord,zcord])

          if resid not in self.store:
            self.store[resid] = {"atom_type": atom_type, "resname": resname}
          self.store[resid][atomname] = [xcord, ycord, zcord]

    print(self.atom_cordinate)
    print(self.hetero_cordinate)

  def listAtoms(self, i):
    atoms = [k for k in self.store[i].keys() if k not in ("atom_type", "resname")]
    return atoms

  def coordAtom(self,i,atomname):
    return self.store[i][atomname]

  def hetero(self):
    for i in self.store:
      if self.store[i].get("atom_type") == "HETATM":
        self.hetero_atom.add(self.store[i].get("resname"))

    return self.hetero_atom

  def distanceAtoms(self,i,atomname_i,j,atomname_j):
    v1 = self.store[i][atomname_i]
    v2 = self.store[j][atomname_j]
    vdiff = [v2[0]-v1[0], v2[1]-v1[1], v2[2]-v1[2]]
    distance = math.sqrt(vdiff[0]**2 + vdiff[1]**2 + vdiff[2]**2)
    return distance


In [ ]:
reader = PdbReader(r"/content/drive/MyDrive/Sem6/proj5 (1).pdb")

Skipping line due to invalid coordinate data: TER    4115      LEU A 540                                                      

[[56.782, -25.22, 15.666], [57.408, -25.36, 17.014], [56.672, -24.501, 18.038], [56.508, -23.285, 17.85], [58.882, -24.972, 16.959], [56.258, -25.148, 19.129], [55.521, -24.502, 20.221], [56.057, -23.127, 20.651], [57.268, -22.87, 20.607], [55.453, -25.444, 21.427], [55.134, -22.255, 21.063], [55.489, -20.915, 21.503], [55.039, -19.85, 20.519], [53.985, -19.235, 20.69], [55.875, -19.637, 19.505], [55.689, -18.678, 18.408], [57.112, -18.321, 18.01], [57.58, -17.194, 18.191], [54.896, -17.421, 18.811], [54.175, -16.65, 17.686], [55.075, -15.66, 16.997], [53.591, -17.609, 16.677], [57.816, -19.347, 17.554], [59.195, -19.237, 17.119], [59.251, -18.697, 15.694], [58.233, -18.632, 15.006], [59.859, -20.625, 17.156], [59.05, -21.562, 16.431], [60.006, -21.103, 18.592], [60.436, -18.253, 15.25], [60.575, -17.729, 13.889], [60.322, -18.839, 12.875], [59.891, -18.577, 1

In [ ]:
print(reader.store)

{4: {'atom_type': 'ATOM', 'resname': 'ASN', 'N': [56.782, -25.22, 15.666], 'CA': [57.408, -25.36, 17.014], 'C': [56.672, -24.501, 18.038], 'O': [56.508, -23.285, 17.85], 'CB': [58.882, -24.972, 16.959]}, 5: {'atom_type': 'ATOM', 'resname': 'ASN', 'N': [56.258, -25.148, 19.129], 'CA': [55.521, -24.502, 20.221], 'C': [56.057, -23.127, 20.651], 'O': [57.268, -22.87, 20.607], 'CB': [55.453, -25.444, 21.427]}, 6: {'atom_type': 'ATOM', 'resname': 'GLY', 'N': [55.134, -22.255, 21.063], 'CA': [55.489, -20.915, 21.503], 'C': [55.039, -19.85, 20.519], 'O': [53.985, -19.235, 20.69]}, 7: {'atom_type': 'ATOM', 'resname': 'LEU', 'N': [55.875, -19.637, 19.505], 'CA': [55.689, -18.678, 18.408], 'C': [57.112, -18.321, 18.01], 'O': [57.58, -17.194, 18.191], 'CB': [54.896, -17.421, 18.811], 'CG': [54.175, -16.65, 17.686], 'CD1': [55.075, -15.66, 16.997], 'CD2': [53.591, -17.609, 16.677]}, 8: {'atom_type': 'ATOM', 'resname': 'THR', 'N': [57.816, -19.347, 17.554], 'CA': [59.195, -19.237, 17.119], 'C': [59.

In [ ]:
reader.listAtoms(542)

['C', 'O', 'OXT', 'CA', 'O3', 'CB']

In [ ]:
reader.coordAtom(4,"CA")

[57.408, -25.36, 17.014]

In [ ]:
reader.hetero()

{' MG', ' MN', 'ATP', 'HOH', 'PYR'}

In [ ]:
reader.distanceAtoms(4,"CA",30,"CA")

27.894772987066947

In [ ]:
# Using the above object and methods find hetero compound having the maximum average
# atomic interactions. For this, first find number of interacting atoms for each hetero-
# compound and divide it by the number of atoms of hetero compound.
# - Two atoms are said to be interacting if distance between two atoms is within 5 Å.

hetero_compounds = reader.hetero()
hetero = {}

for i in reader.store:
  if reader.store[i].get("resname") in hetero_compounds:
    hetero[i] = reader.listAtoms(i)
print(hetero)


{543: ['MN'], 544: ['MG'], 541: ['PG', 'O1G', 'O2G', 'O3G', 'PB', 'O1B', 'O2B', 'O3B', 'PA', 'O1A', 'O2A', 'O3A', "O5'", "C5'", "C4'", "O4'", "C3'", "O3'", "C2'", "O2'", "C1'", 'N9', 'C8', 'N7', 'C5', 'C6', 'N6', 'N1', 'C2', 'N3', 'C4'], 542: ['C', 'O', 'OXT', 'CA', 'O3', 'CB'], 546: ['O'], 547: ['O'], 548: ['O'], 549: ['O']}


In [ ]:
interaction_counts = {}
max_avg = 0
interaction = ""
max_hetero = ""
a=0

for i in hetero:
  a+=1
  total_interactions = 0
  atom_count = len(hetero[i])

  for j in hetero[i]:
    for k in reader.store:
      for l in reader.store[k]:
        if l not in ["atom_type", "resname"]:
          if not (i == k and j == l):
            try:
              d = reader.distanceAtoms(i, j, k, l)
              if d <= 5.0:
                total_interactions += 1
            except:
              continue

  if atom_count > 0:
    avg = total_interactions / atom_count
    resname = reader.store[i]["resname"]
    interaction_counts[resname + str(a)] = avg

    if avg > max_avg:
      max_avg = avg
      interaction = resname

print(f"Hetero compound with max average interactions: {interaction} ({max_avg:.2f})")
print(interaction_counts)

Hetero compound with max average interactions: HOH (34.00)
{' MN1': 26.0, ' MG2': 28.0, 'ATP3': 27.870967741935484, 'PYR4': 20.666666666666668, 'HOH5': 34.0, 'HOH6': 28.0, 'HOH7': 26.0, 'HOH8': 8.0}
